In [1]:
# imports

import pandas as pd
import numpy as np

import spacy
import en_core_web_sm

import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler

In [5]:
# Data load

data_dict = pd.read_csv('../Original_data/data-dictionary.csv')
loans = pd.read_csv('../Original_data/loans.csv', low_memory=False)

In [6]:
data_dict.head()

,variable,description
0,annual_income,The self-reported annual income provided by th...
1,delinquency_2y,The number of 30+ days past-due incidences of ...
2,description,Loan description provided by the borrower.
3,debt_to_income,A ratio calculated using the borrower's total ...
4,earliest_credit_line,The month the borrower's earliest reported cre...


In [7]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237436 entries, 0 to 237435
Data columns (total 32 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   account_id               237436 non-null  int64  
 1   installment              237436 non-null  float64
 2   loan_amount              237436 non-null  float64
 3   interest_rate            237436 non-null  float64
 4   term                     237436 non-null  object 
 5   purpose                  237436 non-null  object 
 6   issue_date               237436 non-null  object 
 7   description              77936 non-null   object 
 8   title                    237423 non-null  object 
 9   home_ownership           237436 non-null  object 
 10  annual_income            237436 non-null  object 
 11  employment_length        228259 non-null  object 
 12  job_title                224835 non-null  object 
 13  earliest_credit_line     237436 non-null  object 
 14  publ

In [8]:
# Target values ratio

ratio = loans["loan_status"].value_counts().to_frame("count")
ratio["percentage"] = (ratio["count"] / len(loans) * 100).round(2)

ratio

,count,percentage
loan_status,,
Fully Paid,206202,86.85
Charged Off,14956,6.30
Late (> 90 days),9298,3.92
Ongoing,5761,2.43
Default,1219,0.51


### Data cleaning 

##### Columns Format Conversion

| Column                | Conversion / Format Description                                  |
|-----------------------|------------------------------------------------------------------|
| **term**              | Convert to category                                          |
| **purpose**           | Convert to category                                          |
| **title**             | **Drop** column - someone notes                                                |
| **home_ownership**    | Convert to category                                          |
| **annual_income**     | Convert to float                                             |
| **employment_length** | Convert to category                                          |
| **job_title**         | Text parsing to map into standardized job title categories   |
| **earliest_credit_line** | Extract first_month + first_year into integer columns |
| **loan_status**       | Convert to category                                          |
| **postcode_district** | Target Encoding                      |
| **district**          | Target Encoding                      |
| **issue_date**        | Extract month + year into integer columns                |


In [9]:
# Convert to float 

loans['annual_income'] = pd.to_numeric(loans['annual_income'], errors='coerce')


In [10]:
# Extract month & first into integer columns

date_val = ['earliest_credit_line', 'issue_date']

for col in date_val:
    # Parse the date column
    loans[col] = pd.to_datetime(loans[col], format='%b-%Y')
    
    # Create new month and year columns (integers)
    loans[f'{col}_month'] = loans[col].dt.month.astype(int)
    loans[f'{col}_year'] = loans[col].dt.year.astype(int)


In [11]:
# Job title parsing with spaCy

# Job titles manual categorization

job_categories = {
    "Management": ["manager", "director", "supervisor", "lead"],
    "Engineering": ["engineer", "technician", "developer", "architect"],
    "Healthcare": ["nurse", "doctor", "physician", "therapist"],
    "Education": ["teacher", "professor", "instructor"],
    "Finance": ["accountant", "analyst", "broker", "consultant"],
    "Sales": ["sales", "associate", "representative"],
    "Customer Service": ["customer", "support", "service"],
    "Operations": ["operator", "warehouse", "logistics"],
    "Legal": ["lawyer", "attorney", "paralegal"],
    "Other": []  # fallback
}


nlp = spacy.load("en_core_web_sm")

def extract_keywords_spacy(title):
    #Extract important nouns/words from a job title using spaC
    if pd.isna(title):
        return []
    doc = nlp(title.lower())
    return [token.lemma_ for token in doc if token.pos_ in ["NOUN", "PROPN"]]

def categorize_job_title(title):
    words = extract_keywords_spacy(title)
    
    for category, keywords in job_categories.items():
        if any(word in keywords for word in words):
            return category
    
    return "Other"

loans["job_title_category"] = loans["job_title"].apply(categorize_job_title)


In [14]:
# Convert to category

cat_val = [
    'term',
    'purpose',
    'home_ownership',
    'employment_length',
    'loan_status',
    'job_title_category'
 ]

for col in cat_val:
    loans[col] = loans[col].astype('category')
    


In [16]:
# Store the dataset for visualisation

loans.to_csv('../Project_csv/visualisation_dataset.csv', index=False)

### Prearing dataset for model fit

In [17]:
# Target Encoding for geo columns

encoder = ce.TargetEncoder(cols=['postcode_district', 'district'])
loans[['postcode_district_te','district_te']] = encoder.fit_transform(
    loans[['postcode_district','district']], 
    loans['loan_status']
)



In [18]:
# Encoded all categories

label_encoders = {}  # store encoders for inverse_transform later

for col in cat_val:
    le = LabelEncoder()
    loans[col + "_enc"] = le.fit_transform(loans[col].astype(str))
    label_encoders[col] = le

In [19]:
# Drop unwanted columns

cols_to_drop = [
    'account_id',
    'issue_date',
    'description',
    'title',
    'earliest_credit_line',
    'job_title',
    'district',
    'postcode_district',
    'term',   # created new encoded columns
    'purpose',
    'home_ownership',
    'employment_length',
    'loan_status',
    'job_title_category'
]

clean_loans = loans.drop(columns=cols_to_drop)

In [20]:
# Normalized data

# Target
target_col = "loan_status_enc"

# Split features and target
X = clean_loans.drop(columns=[target_col])
y = clean_loans [target_col]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Re-create dataframe
X_scaled_loan = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled_loan[target_col] = y


,count,percentage
loan_status,,
Fully Paid,206202,86.85
Charged Off,14956,6.30
Late (> 90 days),9298,3.92
Ongoing,5761,2.43
Default,1219,0.51


In [24]:
ratio_X

,count,percentage
loan_status_enc,,
2,206202,86.85
0,14956,6.30
3,9298,3.92
4,5761,2.43
1,1219,0.51


In [25]:
# drop Ongoing
X_scaled_loan = X_scaled_loan[X_scaled_loan["loan_status_enc"] != 3]

# map to binary
X_scaled_loan["loan_status_enc"] = X_scaled_loan["loan_status_enc"].map(
    {
        2: 0,  # Fully Paid
        0 : 1,  # Charged Off == Default
        3 : 1,  # Late (>90 days) == Default
        1:  1   # Default
    }
)

In [27]:
X_scaled_loan.loan_status_enc.unique()


ratio_X = X_scaled_loan["loan_status_enc"].value_counts().to_frame("count")
ratio_X["percentage"] = (ratio_X["count"] / len(X_scaled_loan) * 100).round(2)

ratio_X

,count,percentage
loan_status_enc,,
0.0,206202,90.38
1.0,16175,7.09


In [28]:
X_scaled_loan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228138 entries, 0 to 237435
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   installment                 228138 non-null  float64
 1   loan_amount                 228138 non-null  float64
 2   interest_rate               228138 non-null  float64
 3   annual_income               226216 non-null  float64
 4   public_records              228138 non-null  float64
 5   last_record_months          28291 non-null   float64
 6   last_delinquency_months     101166 non-null  float64
 7   last_derog_months           43684 non-null   float64
 8   delinquency_2y              228138 non-null  float64
 9   inquiries_6m                228138 non-null  float64
 10  open_accounts               228138 non-null  float64
 11  debt_to_income              228138 non-null  float64
 12  credit_card_usage           227973 non-null  float64
 13  credit_card_balance

In [30]:
# Stored dataset

X_scaled_loan.to_csv('../Project_csv/dataset_for_ML.csv', index=False)